# Dataset preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

## Importing Target variable (S&P Case-Schiller Home Price Index)

In [2]:
df=pd.read_csv('CSUSHPISA.csv')

In [3]:
df.head()

,DATE,CSUSHPISA
0,1987-01-01,63.965
1,1987-02-01,64.424
2,1987-03-01,64.736
3,1987-04-01,65.132
4,1987-05-01,65.563


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   DATE       440 non-null    object 
 1   CSUSHPISA  440 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.0+ KB


In [5]:
df.DATE = pd.to_datetime(df.DATE)

In [6]:
files = ['HNFSEPUSSA', 'HOUST1F', 'HSN1F', 'INTDSRUSM193N',
       'LFACTTTTUSM657S', 'MSACSR', 'NA000334Q', 'NASDAQCOM', 'PERMIT',
       'PERMIT1', 'QUSR628BIS', 'RSAHORUSQ156S', 'TTLCONS', 'UNRATE','CSUSHPISA']

In [7]:
## creating the function to convert the date time format of all the parameters to YYYY-MM-DD

def date_col(arr):
    arr = arr.str.replace(" ","-")
    for i in range(len(arr)):
        arr[i]=dt.datetime.strptime(arr[i], '%d-%m-%y').strftime('%Y-%m-%d')
    return arr

### Merging all the features

In [8]:
# combining all the features into a dataset

for i in range(len(files)-1):
    temp=pd.read_csv(files[i]+'.csv')
    if len(temp.iloc[0,0])==8:
        temp.iloc[:,0] = date_col(temp.iloc[:,0])
    temp.DATE=pd.to_datetime(temp.DATE)
    df=pd.merge(left=df, right=temp, left_on='DATE', right_on='DATE',how='left')

In [9]:
df.head()

,DATE,CSUSHPISA,HNFSEPUSSA,HOUST1F,HSN1F,INTDSRUSM193N,LFACTTTTUSM657S,MSACSR,NA000334Q,NASDAQCOM,PERMIT,PERMIT1,QUSR628BIS,RSAHORUSQ156S,TTLCONS,UNRATE
0,1987-01-01,63.965,356.0,1224,709,5.5,0.197284,6.0,1148876.0,384.227143,1690.0,1088.0,85.4172,63.9,NaN,6.6
1,1987-02-01,64.424,357.0,1282,732,5.5,0.233077,6.2,NaN,411.712632,1689.0,1195.0,NaN,NaN,NaN,6.6
2,1987-03-01,64.736,358.0,1205,713,5.5,0.124242,6.0,NaN,432.204545,1704.0,1132.0,NaN,NaN,NaN,6.6
3,1987-04-01,65.132,358.0,1181,735,5.5,0.055337,6.0,1205023.0,422.771429,1601.0,1057.0,86.2084,63.9,NaN,6.3
4,1987-05-01,65.563,353.0,1141,651,5.5,0.563116,6.7,NaN,416.634000,1500.0,1006.0,NaN,NaN,NaN,6.3


In [10]:
# selecting data from 2003 to 2023
df=df[(df.DATE.dt.year>=2003) & (df.DATE.dt.year<=2023)]

In [11]:
df.reset_index(drop=True,inplace=True)

In [12]:
df.head()

,DATE,CSUSHPISA,HNFSEPUSSA,HOUST1F,HSN1F,INTDSRUSM193N,LFACTTTTUSM657S,MSACSR,NA000334Q,NASDAQCOM,PERMIT,PERMIT1,QUSR628BIS,RSAHORUSQ156S,TTLCONS,UNRATE
0,2003-01-01,128.461,341.0,1537,999,2.25,0.600416,4.0,2743734.0,1389.559524,1808.0,1416.0,114.1698,68.1,863855.0,5.8
1,2003-02-01,129.355,341.0,1301,936,2.25,0.111692,4.5,NaN,1313.261579,1854.0,1357.0,NaN,NaN,859225.0,5.9
2,2003-03-01,130.148,338.0,1399,999,2.25,-0.053388,4.1,NaN,1348.503810,1757.0,1359.0,NaN,NaN,851132.0,5.9
3,2003-04-01,130.884,340.0,1374,1012,2.25,0.309542,4.1,2831413.0,1409.830952,1803.0,1391.0,116.2112,68.2,859459.0,6.0
4,2003-05-01,131.735,343.0,1391,1078,2.25,0.017751,3.9,NaN,1524.182857,1835.0,1389.0,NaN,NaN,866814.0,6.1


## Filling Null values

In [13]:
df.isnull().sum()

DATE                 0
CSUSHPISA            0
HNFSEPUSSA           0
HOUST1F              0
HSN1F                0
INTDSRUSM193N       24
LFACTTTTUSM657S      0
MSACSR               0
NA000334Q          165
NASDAQCOM            0
PERMIT               0
PERMIT1              0
QUSR628BIS         166
RSAHORUSQ156S      165
TTLCONS              0
UNRATE               0
dtype: int64

In [14]:
# forward filling null values for features which has quaterly frquency

df['NA000334Q'] = df['NA000334Q'].fillna(method='ffill')
df['QUSR628BIS'] = df['QUSR628BIS'].fillna(method='ffill')
df['RSAHORUSQ156S'] = df['RSAHORUSQ156S'].fillna(method='ffill')
df['INTDSRUSM193N'] = df['INTDSRUSM193N'].fillna(method='ffill')

In [15]:
# rearranging
df = df[["DATE"]+files]

In [16]:
df.head()

,DATE,HNFSEPUSSA,HOUST1F,HSN1F,INTDSRUSM193N,LFACTTTTUSM657S,MSACSR,NA000334Q,NASDAQCOM,PERMIT,PERMIT1,QUSR628BIS,RSAHORUSQ156S,TTLCONS,UNRATE,CSUSHPISA
0,2003-01-01,341.0,1537,999,2.25,0.600416,4.0,2743734.0,1389.559524,1808.0,1416.0,114.1698,68.1,863855.0,5.8,128.461
1,2003-02-01,341.0,1301,936,2.25,0.111692,4.5,2743734.0,1313.261579,1854.0,1357.0,114.1698,68.1,859225.0,5.9,129.355
2,2003-03-01,338.0,1399,999,2.25,-0.053388,4.1,2743734.0,1348.503810,1757.0,1359.0,114.1698,68.1,851132.0,5.9,130.148
3,2003-04-01,340.0,1374,1012,2.25,0.309542,4.1,2831413.0,1409.830952,1803.0,1391.0,116.2112,68.2,859459.0,6.0,130.884
4,2003-05-01,343.0,1391,1078,2.25,0.017751,3.9,2831413.0,1524.182857,1835.0,1389.0,116.2112,68.2,866814.0,6.1,131.735


In [17]:
df.isnull().sum()

DATE               0
HNFSEPUSSA         0
HOUST1F            0
HSN1F              0
INTDSRUSM193N      0
LFACTTTTUSM657S    0
MSACSR             0
NA000334Q          0
NASDAQCOM          0
PERMIT             0
PERMIT1            0
QUSR628BIS         0
RSAHORUSQ156S      0
TTLCONS            0
UNRATE             0
CSUSHPISA          0
dtype: int64

### Merging some more features

In [18]:
label=pd.read_csv('historical us home prices - Copy.csv')

In [19]:
label.head()

,category,Median Home Price (NSA),CPI-Adjusted Price
0,Thu Jan 15 1953,18080.26,208688.34
1,Sun Feb 15 1953,18080.26,209475.85
2,Sun Mar 15 1953,18080.26,208688.34
3,Wed Apr 15 1953,18122.25,209173.00
4,Fri May 15 1953,18143.25,208631.07


In [20]:
label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847 entries, 0 to 846
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   category                 847 non-null    object 
 1   Median Home Price (NSA)  847 non-null    float64
 2   CPI-Adjusted Price       847 non-null    float64
dtypes: float64(2), object(1)
memory usage: 20.0+ KB


In [21]:
for i in range(len(label)):
    label.iloc[i,0] = label.iloc[i,0][4:] 

In [22]:
label.head()

,category,Median Home Price (NSA),CPI-Adjusted Price
0,Jan 15 1953,18080.26,208688.34
1,Feb 15 1953,18080.26,209475.85
2,Mar 15 1953,18080.26,208688.34
3,Apr 15 1953,18122.25,209173.00
4,May 15 1953,18143.25,208631.07


In [23]:
# formatting date into desired format
for i in range(len(label)):
    label.iloc[i,0]=dt.datetime.strptime(label.iloc[i,0], '%b %d %Y').strftime('%Y-%m-%d')

In [24]:
label.head()

,category,Median Home Price (NSA),CPI-Adjusted Price
0,1953-01-15,18080.26,208688.34
1,1953-02-15,18080.26,209475.85
2,1953-03-15,18080.26,208688.34
3,1953-04-15,18122.25,209173.00
4,1953-05-15,18143.25,208631.07


In [25]:
# resetting the date to start of each month to ease the merging process
label.iloc[:,0] = label.iloc[:,0].str.replace("-15","-01")

In [26]:
label.head()

,category,Median Home Price (NSA),CPI-Adjusted Price
0,1953-01-01,18080.26,208688.34
1,1953-02-01,18080.26,209475.85
2,1953-03-01,18080.26,208688.34
3,1953-04-01,18122.25,209173.00
4,1953-05-01,18143.25,208631.07


In [27]:
label.isnull().sum()

category                   0
Median Home Price (NSA)    0
CPI-Adjusted Price         0
dtype: int64

In [28]:
label.category=pd.to_datetime(label.category)

In [29]:
# final dataset 
df1=pd.merge(left=df, right=label, left_on='DATE', right_on='category',how='left')

In [30]:
df1.isnull().sum()

DATE                       0
HNFSEPUSSA                 0
HOUST1F                    0
HSN1F                      0
INTDSRUSM193N              0
LFACTTTTUSM657S            0
MSACSR                     0
NA000334Q                  0
NASDAQCOM                  0
PERMIT                     0
PERMIT1                    0
QUSR628BIS                 0
RSAHORUSQ156S              0
TTLCONS                    0
UNRATE                     0
CSUSHPISA                  0
category                   1
Median Home Price (NSA)    1
CPI-Adjusted Price         1
dtype: int64

In [31]:
# dropping unnecessary column
df1.drop(columns=['category'], inplace=True)

## Final dataset

In [32]:
df1.head()

,DATE,HNFSEPUSSA,HOUST1F,HSN1F,INTDSRUSM193N,LFACTTTTUSM657S,MSACSR,NA000334Q,NASDAQCOM,PERMIT,PERMIT1,QUSR628BIS,RSAHORUSQ156S,TTLCONS,UNRATE,CSUSHPISA,Median Home Price (NSA),CPI-Adjusted Price
0,2003-01-01,341.0,1537,999,2.25,0.600416,4.0,2743734.0,1389.559524,1808.0,1416.0,114.1698,68.1,863855.0,5.8,128.461,160883.13,271850.87
1,2003-02-01,341.0,1301,936,2.25,0.111692,4.5,2743734.0,1313.261579,1854.0,1357.0,114.1698,68.1,859225.0,5.9,129.355,161921.15,271512.85
2,2003-03-01,338.0,1399,999,2.25,-0.053388,4.1,2743734.0,1348.503810,1757.0,1359.0,114.1698,68.1,851132.0,5.9,130.148,163045.67,271765.80
3,2003-04-01,340.0,1374,1012,2.25,0.309542,4.1,2831413.0,1409.830952,1803.0,1391.0,116.2112,68.2,859459.0,6.0,130.884,164583.47,274926.03
4,2003-05-01,343.0,1391,1078,2.25,0.017751,3.9,2831413.0,1524.182857,1835.0,1389.0,116.2112,68.2,866814.0,6.1,131.735,166198.17,278077.16


In [33]:
df1.tail()

,DATE,HNFSEPUSSA,HOUST1F,HSN1F,INTDSRUSM193N,LFACTTTTUSM657S,MSACSR,NA000334Q,NASDAQCOM,PERMIT,PERMIT1,QUSR628BIS,RSAHORUSQ156S,TTLCONS,UNRATE,CSUSHPISA,Median Home Price (NSA),CPI-Adjusted Price
243,2023-04-01,430.0,847,679,0.25,-0.025790,7.6,6802375.0,12073.464211,1417.0,856.0,157.8759,65.9,1907837.0,3.4,300.198,391534.51,396262.15
244,2023-05-01,426.0,1012,710,0.25,0.077990,7.2,6802375.0,12446.530455,1496.0,902.0,157.8759,65.9,1946733.0,3.7,302.620,396887.99,400671.21
245,2023-06-01,429.0,930,683,0.25,0.079728,7.5,6802375.0,13463.777619,1441.0,924.0,157.8759,65.9,1956226.0,3.6,304.651,399117.81,401625.47
246,2023-07-01,431.0,988,736,0.25,0.091045,7.0,6920709.0,14033.091000,1443.0,930.0,157.8759,66.0,1969005.0,3.5,306.634,401559.07,403312.74
247,2023-08-01,432.0,933,676,0.25,0.440447,7.7,6920709.0,13743.213043,1541.0,948.0,157.8759,66.0,1988312.0,3.8,309.404,NaN,NaN


In [34]:
df1.isnull().sum()

DATE                       0
HNFSEPUSSA                 0
HOUST1F                    0
HSN1F                      0
INTDSRUSM193N              0
LFACTTTTUSM657S            0
MSACSR                     0
NA000334Q                  0
NASDAQCOM                  0
PERMIT                     0
PERMIT1                    0
QUSR628BIS                 0
RSAHORUSQ156S              0
TTLCONS                    0
UNRATE                     0
CSUSHPISA                  0
Median Home Price (NSA)    1
CPI-Adjusted Price         1
dtype: int64

In [35]:
df1['Median Home Price (NSA)'] = df1['Median Home Price (NSA)'].fillna(method='ffill')
df1['CPI-Adjusted Price'] = df1['CPI-Adjusted Price'].fillna(method='ffill')

In [36]:
df1.isnull().sum()

DATE                       0
HNFSEPUSSA                 0
HOUST1F                    0
HSN1F                      0
INTDSRUSM193N              0
LFACTTTTUSM657S            0
MSACSR                     0
NA000334Q                  0
NASDAQCOM                  0
PERMIT                     0
PERMIT1                    0
QUSR628BIS                 0
RSAHORUSQ156S              0
TTLCONS                    0
UNRATE                     0
CSUSHPISA                  0
Median Home Price (NSA)    0
CPI-Adjusted Price         0
dtype: int64

In [37]:
df1.columns

Index(['DATE', 'HNFSEPUSSA', 'HOUST1F', 'HSN1F', 'INTDSRUSM193N',
       'LFACTTTTUSM657S', 'MSACSR', 'NA000334Q', 'NASDAQCOM', 'PERMIT',
       'PERMIT1', 'QUSR628BIS', 'RSAHORUSQ156S', 'TTLCONS', 'UNRATE',
       'CSUSHPISA', 'Median Home Price (NSA)', 'CPI-Adjusted Price'],
      dtype='object')

In [38]:
df1= df1[['DATE', 'HNFSEPUSSA', 'HOUST1F', 'HSN1F', 'INTDSRUSM193N',
       'LFACTTTTUSM657S', 'MSACSR', 'NA000334Q', 'NASDAQCOM', 'PERMIT',
       'PERMIT1', 'QUSR628BIS', 'RSAHORUSQ156S', 'TTLCONS', 'UNRATE', 'Median Home Price (NSA)', 'CPI-Adjusted Price','CSUSHPISA']]

In [39]:
# saving dataset to csv 
df1.to_csv('final_data_frame.csv', index= False)